<a href="https://colab.research.google.com/github/Enmasdiaz1/AgendaAPP_Csharp-Windows-Form/blob/master/Copy_of_Proyecto_Puzzle_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Librerías 

import argparse
import timeit
import resource
from collections import deque
from heapq import heappush, heappop, heapify
import itertools

In [ ]:
# clase estado

class State:

    def __init__(self, state, parent, move, depth, cost, key):

        self.state = state

        self.parent = parent

        self.move = move

        self.depth = depth

        self.cost = cost

        self.key = key

        if self.state:
            self.map = ''.join(str(e) for e in self.state)

    def __eq__(self, other):
        return self.map == other.map

    def __lt__(self, other):
        return self.map < other.map


In [ ]:
#Variables

goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
goal_node = State
initial_state = list()
board_len = 0
board_side = 0

nodes_expanded = 0
max_search_depth = 0
max_frontier_size = 0

moves = list()
costs = set()

In [ ]:
#Algoritmo bfs

def bfs(start_state):

#se declaran las variables
    global max_frontier_size, goal_node, max_search_depth

#se setea la variable explored y queue "se utliza set porque es mas eficiente, mas rapido no permite elementos duplicados"
    explored, queue = set(), deque([State(start_state, None, None, 0, 0, 0)])

    while queue:

        node = queue.popleft()

        explored.add(node.map)

        if node.state == goal_state:
            goal_node = node
            return queue

        neighbors = expand(node)

        for neighbor in neighbors:
            if neighbor.map not in explored:
                queue.append(neighbor)
                explored.add(neighbor.map)

                if neighbor.depth > max_search_depth:
                    max_search_depth += 1

        if len(queue) > max_frontier_size:
            max_frontier_size = len(queue)


            


In [ ]:
#Algoritmo dfs

def dfs(start_state):

    global max_frontier_size, goal_node, max_search_depth

    explored, stack = set(), list([State(start_state, None, None, 0, 0, 0)])

    while stack:

        node = stack.pop()

        explored.add(node.map)

        if node.state == goal_state:
            goal_node = node
            return stack

        neighbors = reversed(expand(node))

        for neighbor in neighbors:
            if neighbor.map not in explored:
                stack.append(neighbor)
                explored.add(neighbor.map)

                if neighbor.depth > max_search_depth:
                    max_search_depth += 1

        if len(stack) > max_frontier_size:
            max_frontier_size = len(stack)


In [ ]:
#Algoritmo ast

"""
Luis Alberto Marmol
Yaher Hichez Garcia
"""

#El algoritmo A* o AST/A estrella es un algoritmo que trata de obtener el resultado a menor costo de distancia entre el nodo objetivo y el origen

#La funcion ast decibe el estado origen del arreglo
def ast(start_state):

#en esta parte se declaran las variables globales, la maxima longitud de frontera, el noto meta y la profundidad de busqueda
    global max_frontier_size, goal_node, max_search_depth

#se crea una lista para los campos explorados, dos para heapsort y un contador
    explored, heap, heap_entry, counter = set(), list(), {}, itertools.count()

#llave sera igual al estado origen
    key = h(start_state)

#se declara root y se le asigna una coleccion desde su estado inicial hasta los estados actuales
    root = State(start_state, None, None, 0, 0, key)

#Esta es la coleccion de entradas de los estados, los cuales son pasados para llegar al resultado
    entry = (key, 0, root)

    heappush(heap, entry)

#se procede a un mapeado a los estados segun su entrada
    heap_entry[root.map] = entry

#Este es el bucle de heapsearch, este lo compara y lo ordena
    while heap:
#aqui el nodo avanzara segun la coleccion de movimientos realizados
        node = heappop(heap)

#Los explorados seran agregados y apuntando la visita o movimiento realizado
        explored.add(node[2].map)

#mientras mas profundo el nodo sera mas cerca del resultado
        if node[2].state == goal_state:
            goal_node = node[2]
            return heap

#Los vecinos realizaran la comprobacion segun la profundidad o acercamiento

#Expand pertenece a sympy
        neighbors = expand(node[2])

        for neighbor in neighbors:

            neighbor.key = neighbor.cost + h(neighbor.state)

            entry = (neighbor.key, neighbor.move, neighbor)

            #Determinar si el vecino no fue explorado
            #Si no fue explorado 
            if neighbor.map not in explored:

                heappush(heap, entry)

                explored.add(neighbor.map)
#Se mapeara los vecios en la coleccion de entrada o los movimientos hechos y asi validar la visita

                heap_entry[neighbor.map] = entry

#Se determina si la profundidad del vecino es mayor que la profundidad maxima de busqueda, de ser asi se incrementara en 1
                if neighbor.depth > max_search_depth:
                    max_search_depth += 1

#De lo contrario se mapeara el vecino para validad la entrada y saber si se ha encontrado una posicion correcta
            elif neighbor.map in heap_entry and neighbor.key < heap_entry[
                    neighbor.map][2].key:

                hindex = heap.index((heap_entry[neighbor.map][2].key,
                                     heap_entry[neighbor.map][2].move,
                                     heap_entry[neighbor.map][2]))

                heap[int(hindex)] = entry
                

                heap_entry[neighbor.map] = entry

                heapify(heap)
#Ya luego se evalua la coleccion para establecer la posicion correcta, y asi establecer la frontera y resolucion de busqueda
        if len(heap) > max_frontier_size:
            max_frontier_size = len(heap)


In [ ]:
#Expandir nodos

def expand(node):

    global nodes_expanded
    nodes_expanded += 1

    neighbors = list()

    neighbors.append(
        State(move(node.state, 1), node, 1, node.depth + 1, node.cost + 1, 0))
    neighbors.append(
        State(move(node.state, 2), node, 2, node.depth + 1, node.cost + 1, 0))
    neighbors.append(
        State(move(node.state, 3), node, 3, node.depth + 1, node.cost + 1, 0))
    neighbors.append(
        State(move(node.state, 4), node, 4, node.depth + 1, node.cost + 1, 0))

    nodes = [neighbor for neighbor in neighbors if neighbor.state]

    return nodes


In [ ]:
# movimientos

def move(state, position):

    new_state = state[:]

    index = new_state.index(0)

    if position == 1:  # Up

        if index not in range(0, board_side):

            temp = new_state[index - board_side]
            new_state[index - board_side] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

    if position == 2:  # Down

        if index not in range(board_len - board_side, board_len):

            temp = new_state[index + board_side]
            new_state[index + board_side] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

    if position == 3:  # Left

        if index not in range(0, board_len, board_side):

            temp = new_state[index - 1]
            new_state[index - 1] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None

    if position == 4:  # Right

        if index not in range(board_side - 1, board_len, board_side):

            temp = new_state[index + 1]
            new_state[index + 1] = new_state[index]
            new_state[index] = temp

            return new_state
        else:
            return None


In [ ]:
# altura

def h(state):

    return sum(
        abs(b % board_side - g % board_side) +
        abs(b // board_side - g // board_side)
        for b, g in ((state.index(i), goal_state.index(i))
                     for i in range(1, board_len)))


In [ ]:
# backtrace

def backtrace():

    current_node = goal_node

    while initial_state != current_node.state:

        if current_node.move == 1:
            movement = 'Up'
        elif current_node.move == 2:
            movement = 'Down'
        elif current_node.move == 3:
            movement = 'Left'
        else:
            movement = 'Right'

        moves.insert(0, movement)
        current_node = current_node.parent

    return moves


In [ ]:
# exportar

def export(frontier, time):

    global moves

    moves = backtrace()

    file = open('output.txt', 'w')
    file.write("path_to_goal: " + str(moves))
    file.write("\ncost_of_path: " + str(len(moves)))
    file.write("\nnodes_expanded: " + str(nodes_expanded))
    file.write("\nsearch_depth: " + str(goal_node.depth))
    file.write("\nmax_search_depth: " + str(max_search_depth))
    file.write("\nrunning_time: " + format(time, '.8f'))
    file.write("\nmax_ram_usage: " + format(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000.0, '.8f'))
    file.close()


In [ ]:
# Leer

def read(configuration):

    global board_len, board_side

    data = configuration.split(",")

    for element in data:
        initial_state.append(int(element))

    board_len = len(initial_state)

    board_side = int(board_len**0.5)


In [ ]:
# dls_mod

def dls_mod(start_state, threshold):

    global max_frontier_size, goal_node, max_search_depth, costs

    explored, stack = set(), list(
        [State(start_state, None, None, 0, 0, threshold)])

    while stack:

        node = stack.pop()

        explored.add(node.map)

        if node.state == goal_state:
            goal_node = node
            return stack

        if node.key > threshold:
            costs.add(node.key)

        if node.depth < threshold:

            neighbors = reversed(expand(node))

            for neighbor in neighbors:
                if neighbor.map not in explored:

                    neighbor.key = neighbor.cost + h(neighbor.state)
                    stack.append(neighbor)
                    explored.add(neighbor.map)

                    if neighbor.depth > max_search_depth:
                        max_search_depth += 1

            if len(stack) > max_frontier_size:
                max_frontier_size = len(stack)

    return min(costs)




In [ ]:
# clase principal

def main():

    parser = argparse.ArgumentParser()

    parser.add_argument('algorithm')
    parser.add_argument('board')
    args = parser.parse_args()

    read(args.board)

    function = function_map[args.algorithm]

    start = timeit.default_timer()

    frontier = function(initial_state)

    stop = timeit.default_timer()

    export(frontier, stop - start)


In [ ]:
# funcion que verifica que algoritmo introdujo el usuario por consola

function_map = {'bfs': bfs, 'dfs': dfs, 'ast': ast}

if __name__ == '__main__':
    main()

In [ ]:
# Necesario ejecutarlo para habilitar la consola

from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os
def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [ ]:
#@title Colab Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });